In [10]:
# connection
import datetime
from random import random, randint

import requests
import yaml
from elasticsearch.helpers import bulk, parallel_bulk
from elasticsearch import Elasticsearch


config = yaml.load(open("/opt/configs/360/yunjia.yml", 'rb').read(), Loader=yaml.CLoader)

# api = 'http://xxx:8200/'  #general v2
# api = 'http://xxx:8200/'  # general v3

api = config['elasticsearch']['bushu-1']['api']
es = Elasticsearch([api])
es.info()

{'name': '1d449827cfa6',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'X3dZ4RPMTa-2av-DftNkfQ',
 'version': {'number': '7.8.1',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': 'b5ca9c58fb664ca8bf9e4057fc229b3396bf3a89',
  'build_date': '2020-07-21T16:40:44.668009Z',
  'build_snapshot': False,
  'lucene_version': '8.5.1',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [ ]:
# get current cluster settings

requests.get(api + '_cluster/settings?include_defaults=true').json()

In [ ]:
# cluster settings
# https://www.elastic.co/guide/en/elasticsearch/reference/6.8/disk-allocator.html

url = api + '_cluster/settings'
data = {
    "transient": {
#         "cluster.routing.allocation.cluster_concurrent_rebalance": 4,  # default 2
#         "cluster.routing.allocation.node_concurrent_recoveries": 4,  # default 2
#         "indices.recovery.max_bytes_per_sec": "200mb",  # default 40mb
        "cluster.routing.allocation.disk.threshold_enabled": True,  # default true
        "cluster.routing.allocation.disk.watermark.low": "90%",  # default 85%, stop new replica shards
        "cluster.routing.allocation.disk.watermark.high": "98%",  # default 90%, stop all new shards
        "cluster.routing.allocation.disk.watermark.flood_stage": "99%",  # default 95%, readonly
#         "cluster.routing.allocation.enable": "primary"
    }
}

r = requests.put(url, json=data)
r.json()

In [ ]:
# bulk
import json
import gzip

url = api+"perf-spring-logs-write/logs/_bulk"
d = {"@timestamp": "2018-12-07T05:00:00.000+08:00", "app": "test", "message": "tt2"}
header = {"Content-encoding": "gzip"}

cnt = ""
for _ in range(700):
    cnt += json.dumps({"index": {}}) + '\n'
    cnt += json.dumps(d) + "\n"
    
r = requests.post(url, headers=header, data=gzip.compress(cnt.encode('utf8')))
print(r.status_code)
# r.json()

In [ ]:
r.json()

In [ ]:
import datetime

url = api+"test/doc"

data = {
    "text2": 123,
}

r = requests.post(url, ,json=data)
print(r.status_code)
print(r.text)
print(r.json())

In [ ]:
uri = '_cluster/health'
uri = '_cluster/pending_tasks'
uri = '_cat/nodes?v'
# uri = '_node/stats/jvm?pretty'
# uri = '_node/stats/process?pretty'
# uri = '_node/hot_threads?human=true'
# uri = '_tasks'
uri = '_stats'

# requests.get(api+uri).json()
requests.get(api+uri).json()['indices']['uat-spark-logs']['primaries']['docs']

In [ ]:
# indexes

def delete_all_indexes():
    for idx in es.indices.get_alias('*').keys():
        if idx.startswith('.') \
                or idx in ('fluentd', 'test', 'simpleauth'):
            continue
            
        if idx.startswith('zipkin_') or idx.startswith('zipkin-'):
            print(f"{idx} delete_index(idx)")
        
            
def show_all_indexes():
    return list(es.indices.get_alias('*').keys())


def delete_index(index):
    return es.indices.delete(index=index)



delete_all_indexes()
# delete_index('sit-cp-logs')
# delete_index('sit-spring-logs')
# delete_index('uat-spring-logs')
# delete_index('perf-spring-logs')
# delete_index('prod-spring-logs')
sorted(show_all_indexes())


In [ ]:
for idx in show_all_indexes():
    if idx.startswith('zipkin:'):
        delete_index(idx)

In [ ]:
import re

reg = re.compile('.*-write')
for idx in show_all_indexes():
    if reg.match(idx):
        print('delete', idx)
        delete_index(idx)

In [ ]:
# delete documents
body = {
    'query': {
        'match_all': {}
    },
    'size': '100000',
}

def delete_docus_in_index(index, type_):
    r = requests.post(api+'{}/{}/_delete_by_query?pretty'.format(index, type_), json=body)
    return r.json()
    
    
delete_docus_in_index('sit-connector-logs', 'logs')
# delete_docus_in_index('uat-spark-logs', 'logs')

In [ ]:
# create index
# no need for rollover

def create_index(index):
    data = {
        'settings': {
            'number_of_shards': 3,
            'number_of_replicas': 1,
            "index.store.type": "niofs",
        }
    }
    r = requests.put(api+index, json=data, timeout=3)
    print(r.json())

# create_index('logs')
    
# index
# create_index(index='sit-spring-logs-v3')
# create_index(index='sit-spark-logs')
# create_index(index='sit-cp-logs')
# create_index(index='sit-geely-logs')
# create_index(index='sit-gateway-logs')
# create_index(index='sit-connector-logs')
# create_index(index='sit-emqtt-logs')
# create_index(index='sit-bigdata-logs')
create_index(index='sit-speech-logs')

# uat
# create_index(index='uat-spring-logs-v2')
# create_index(index='uat-spark-logs')
# create_index(index='uat-cp-logs')
# create_index(index='uat-gateway-logs')
# create_index(index='uat-connector-logs')
# create_index(index='uat-emqtt-logs')


# perf
# create_index(index='perf-spring-logs-v2')
# create_index(index='perf-spark-logs')
# create_index(index='perf-cp-logs')
# create_index(index='perf-geely-logs')
# create_index(index='perf-gateway-logs')
# create_index(index='perf-connector-logs')
# create_index(index='perf-emqtt-logs')


# prod
# create_index(index='prod-spring-logs-v2')
# create_index(index='prod-spark-logs')
# create_index(index='prod-cp-logs')
# create_index(index='prod-gateway-logs')
# create_index(index='prod-connector-logs')
# create_index(index='prod-geely-logs')
# create_index(index='prod-emqtt-logs')
# create_index(index='prod-bigdata-logs')
create_index(index='prod-speech-logs')

# test
# create_index(index='test')
# create_index(index='es-stats')

In [ ]:
# put spark mapping

def create_mapping(index, type_):
    url = '{}{}/_mapping/{}'.format(api, index, type_)
    data = {
        "properties": {
#             "time": {
#                 "type": "date",
#                 "format": "yyyy-MM-dd HH:mm:ss Z"
#             },
            "level": {
                "type": "keyword",
                "index": "not_analyzed",
                "doc_values": True,
            },
            "app_info": {
                "type": "keyword",
                "index": "not_analyzed",
                "doc_values": True,
            },
            "message": {
                "type": "string",
            },
        }
    }

    r = requests.put(url, json=data)
    print(index, r.text)
    return r.status_code


# print(create_mapping('perf-spark-logs', 'logs'))
# print(create_mapping('uat-spark-logs', 'logs'))
print(create_mapping('sit-bigdata_wuling-logs', 'logs'))
# print(create_mapping('prod-spark-logs', 'logs'))
# print(create_mapping('test', 'logs'))

In [ ]:
# put bigdata mapping

def create_mapping(index, type_):
    url = '{}{}/_mapping/{}'.format(api, index, type_)
    data = {
#         "properties": {
#             "happentime": {
#                 "type": "date",
#                 "format": "yyyy-MM-ddTHH:mm:ss"
#             },
#             "level": {
#                 "type": "keyword",
#                 "index": "not_analyzed",
#                 "doc_values": True,
#             },
#             "app_info": {
#                 "type": "keyword",
#                 "index": "not_analyzed",
#                 "doc_values": True,
#             },
#             "message": {
#                 "type": "string",
#             },
        }
    }

    r = requests.put(url, json=data)
    print(index, r.text)
    return r.status_code


print(create_mapping('sit-bigdata-logs', 'logs'))
print(create_mapping('prod-bigdata-logs', 'logs'))


In [ ]:
# put spring mapping

def create_mapping(index, type_):
    url = '{}{}/_mapping/{}'.format(api, index, type_)
    data = {
        "properties": {
            "level": {
                "type": "keyword",
                "index": "not_analyzed",
                "doc_values": True,
            },
            "app": {
                "type": "keyword",
                "index": "not_analyzed",
                "doc_values": True,
            },
            "class": {
                "type": "keyword",
                "index": "not_analyzed",
                "doc_values": True,
            },
            "thread": {
                "type": "keyword",
                "index": "not_analyzed",
                "doc_values": True,
            },
            "line": {
                "type": "short",
            },
            "message": {
                "type": "string",
            },
            "datasource": {
                "type": "keyword",
                "index": "not_analyzed",
                "doc_values": True,
            }
        }
    }

    r = requests.put(url, json=data)
    print(index, r.text)
    return r.status_code


# print(create_mapping('perf-spring-logs-v2', 'logs'))
# print(create_mapping('uat-spring-logs-v2', 'logs'))
print(create_mapping('sit-spring-logs-v3', 'logs'))
# print(create_mapping('prod-spring-logs-v2', 'logs'))
# print(create_mapping('test', 'logs'))

In [ ]:
# put gateway mapping

def create_mapping(index, type_):
    url = '{}{}/_mapping/{}'.format(api, index, type_)
    data = {
        "properties": {
            "level": {
                "type": "keyword",
                "index": "not_analyzed",
                "doc_values": True,
            },
            "app": {
                "type": "keyword",
                "index": "not_analyzed",
                "doc_values": True,
            },
            "class": {
                "type": "keyword",
                "index": "not_analyzed",
                "doc_values": True,
            },
            "thread": {
                "type": "keyword",
                "index": "not_analyzed",
                "doc_values": True,
            },
            "line": {
                "type": "short",
            },
            "message": {
                "type": "string",
            },
        }
    }

    r = requests.put(url, json=data)
    print(index, r.text)
    return r.status_code


print(create_mapping('perf-gateway-logs', 'logs'))
print(create_mapping('uat-gateway-logs', 'logs'))
print(create_mapping('sit-gateway-logs', 'logs'))
print(create_mapping('prod-gateway-logs', 'logs'))
# print(create_mapping('test', 'logs'))

In [ ]:
# put connector mapping

def create_mapping(index, type_):
    url = '{}{}/_mapping/{}'.format(api, index, type_)
    data = {
        "properties": {
            "level": {
                "type": "keyword",
                "index": "not_analyzed",
                "doc_values": True,
            },
            "app": {
                "type": "keyword",
                "index": "not_analyzed",
                "doc_values": True,
            },
            "class": {
                "type": "keyword",
                "index": "not_analyzed",
                "doc_values": True,
            },
            "thread": {
                "type": "keyword",
                "index": "not_analyzed",
                "doc_values": True,
            },
            "line": {
                "type": "short",
            },
            "message": {
                "type": "string",
            },
        }
    }

    r = requests.put(url, json=data)
    print(index, r.text)
    return r.status_code


print(create_mapping('perf-connector-logs', 'logs'))
print(create_mapping('uat-connector-logs', 'logs'))
print(create_mapping('sit-connector-logs', 'logs'))
print(create_mapping('prod-connector-logs', 'logs'))
# print(create_mapping('test', 'logs'))

In [ ]:
# put geely mapping

def create_mapping(index, type_):
    url = '{}{}/_mapping/{}'.format(api, index, type_)
    data = {
        "properties": {
#             "time": {
#                 "type": "date",
#                 "format": "yyyy-MM-dd HH:mm:ss.SSS Z"
#             },
            "level": {
                "type": "keyword",
                "index": "not_analyzed",
                "doc_values": True,
            },
            "class": {
                "type": "keyword",
                "index": "not_analyzed",
                "doc_values": True,
            },
            "thread": {
                "type": "keyword",
                "index": "not_analyzed",
                "doc_values": True,
            },
            "project": {
                "type": "keyword",
                "index": "not_analyzed",
                "doc_values": True,
            },
            "line": {
                "type": "integer",
            },
            "message": {
                "type": "string",
            },
        }
    }

    r = requests.put(url, json=data)
    print(index, r.text)
    return r.status_code


print(create_mapping('test', 'logs'))
print(create_mapping('sit-geely-logs', 'logs'))
print(create_mapping('perf-geely-logs', 'logs'))
print(create_mapping('prod-geely-logs-v2', 'logs'))
# print(create_mapping('test', 'logs'))

In [ ]:
# cp mapping

def create_mapping(index, type_):
    url = '{}{}/_mapping/{}'.format(api, index, type_)
    data = {
        "properties": {
            "level": {
                "type": "keyword",
                "index": "not_analyzed",
                "doc_values": True,
            },
            "class": {
                "type": "keyword",
                "index": "not_analyzed",
                "doc_values": True,
            },            "message": {
                "type": "string",
            },
            "datasource": {
                "type": "keyword",
                "index": "not_analyzed",
                "doc_values": True,
            }
        }
    }

    r = requests.put(url, json=data)
    print(index, r.text)
    return r.status_code


print(create_mapping('sit-cp-logs', 'logs'))
print(create_mapping('uat-cp-logs', 'logs'))
print(create_mapping('perf-cp-logs', 'logs'))
print(create_mapping('prod-cp-logs', 'logs'))
# print(create_mapping('test', 'logs'))

In [ ]:
# put test mapping

def create_mapping(index, type_):
    url = '{}{}/_mapping/{}'.format(api, index, type_)
    data = {
        "properties": {
            "message": {
                "type": "string",
            },
        },
        "_ttl": {
            "enabled": True,
            "default": "1s",
        }
    }

    r = requests.put(url, json=data)
    print(r.text)
    return r.status_code



print(create_mapping('test', 'test'))

In [ ]:
# reindex
url = api+'_reindex'

def reindex(fr_idx, to_idx):
    data = {
    #     'size': 100000,
        "conflicts": "proceed",
        'source': {
            'remote': {
                'host': 'http://1.1.1.1:8200/',
                'username': 'superuser',
                'password': 'xxx',
            },        
            'index': fr_idx,
            'type': 'stats',
        },
        'dest': {
            'index': to_idx,
    #         "op_type": "create",
        }
    }

    r = ''
    try:
        r = requests.post(url, json=data, timeout=3)
        r = r.json()
    except requests.Timeout:
        pass
    
    print(fr_idx, r)
    
    
# reindex
reindex('monitor-stats-alias', 'monitor-stats-write')
# reindex('uat-spark-logs-write', 'uat-spark-logs-2018.05.29-1')
# reindex('perf-spark-logs-write', 'perf-spark-logs-2018.05.29-1')
# reindex('prod-spark-logs-write', 'prod-spark-logs-2018.05.29-1')

In [ ]:
# reindex batch

import re

source_api = 'http://xxx:8999/1037308040/'
dest_api = 'http://xxx:8200/'

idx_reg = re.compile('\w+-\w+-\w+-.*')


def reindex_batch(idx):
    url = api+'_reindex'
    data = {
        'conflicts': 'proceed',
        'source': {
            'remote': {
                'host': source_api,
            },
            'index': idx,
            'type': 'logs',
        },
        'dest': {
            'index': idx
        }
    }
    try:
        print('reindex for {}'.format(idx))
        r = requests.post(url, json=data, timeout=3)
    except requests.Timeout:
        pass
    
    print("all done")
    
    
    
def get_all_indices():
    url = source_api+"_cat/indices/?h=index&format=json"
    r = requests.get(url).json()
    return filter(lambda v: idx_reg.match(v), map(lambda d: d['index'], r))
        
        
for idx in get_all_indices():
#     print(idx)
    reindex_batch(idx)

In [ ]:
# alias
# https://www.elastic.co/guide/en/elasticsearch/reference/6.2/indices-aliases.html


url = api+"_aliases"
print(url)

def create_alias(index, alias, f=None):
    data = {
        'actions': [
            {"add": {
                "index": index, 
                "alias": alias,
            }}
        ]
    }
    
    if f:
        data['actions'][0]['add']['filter'] = f

    r = requests.post(url, json=data)
    print(index, r.status_code,r.json())
    
    
# alias
# create_alias('sit-spark-logs-2018.05.29-1', 'sit-spark-logs-write')
# create_alias('uat-spark-logs-2018.05.29-1', 'uat-spark-logs-write')
# create_alias('perf-spark-logs-2018.05.29-1', 'perf-spark-logs-write')
# create_alias('prod-spark-logs-2018.05.29-1', 'prod-spark-logs-write')
create_alias('zipkin_prod:span*', 'zipkin-prod-alias')

# write-alias



In [ ]:
# user, # auth

indexes = [
    '/.kibana',
    '/es-stats',
    '/perf-cp-logs',
    '/perf-spark-logs',
    '/perf-spring',
    '/perf-spring-logs',
    '/prod-cp-logs',
    '/prod-spark-logs',
    '/prod-spring-logs',
    '/simpleauth',
    '/sit-cp-logs',
    '/sit-spark-logs',
    '/sit-spring-logs',
    '/uat-cp-logs',
    '/uat-spark-logs',
    '/uat-spring-logs',
]

indexes = ["sit-speech-logs-alias", "prod-speech-logs-alias"]

url = '{}_user/create'.format(api)
# url = api + '_user/delete'
# url = '{}_user/alter'.format(api)
data = {
    'username': 'speech',
    'password': 'xxx',
#     'ip_whitelist': ['172.16.4.*'],
    'ip_whitelist': ['*'],
    'get_path': ['/'], #+indexes,
    'head_path': ['/'], #+indexes,
    'post_path': ['/'], #+indexes,
    "put_path": ["/"],
    'del_path': ['/'],
    'hostname_whitelist': ['*'],
#     'read_index': ["*"],
#     'write_index': ["*"],
    'read_index': ["sit-speech-logs-alias", "prod-speech-logs-alias", ".kibana"],
    'write_index': [".kibana"],
}

r = requests.post(url, json=data)
r.json()

In [ ]:
# zipkin users


url = "{}_user/create".format(api)
# url = api + '_user/delete'
# url = '{}_user/alter'.format(api)
data = {
    "username": "123",
    "password": "123",
    "ip_whitelist": ["*"],
    "get_path": ["/", "/_", "/zipkin_prod:"],
    "head_path": ["/", "/_", "/zipkin_prod:"],
    "put_path": ["/_", "/zipkin_prod:"],
    "post_path": ["/zipkin_prod:", "/_"],
    "hostname_whitelist": ["*"],
    "read_index": ["*"],
    "write_index": ["*"],
}

r = requests.post(url, json=data)
r.json()


In [ ]:
# show user
url = '{}_user/show'.format(api)
requests.post(url).json()

In [ ]:
# bulk insert doucuments


def generate_level():
    if random() < 0.8:
        return 'INFO'
    else:
        return 'ERROR'
    
    
def generate_time():
    ts = (datetime.datetime.now() - datetime.timedelta(hours=randint(10, 100))).isoformat(timespec='milliseconds')
    return '{}Z'.format(ts)
    
    

def generate_data(index, type_, n):
    for _ in range(n):
        l = generate_level()
        t = generate_time()
#         print(t)
        yield {
            '_index': index,
            '_type': type_,
            '_source': {
                'level': l,
                'message': '{} {} xxx'.format(t, l),
                '@timestamp': t,
            }
        }
        
        

def insert_docus(index, type_, n):
    for resp in parallel_bulk(es, generate_data(index, type_, n), thread_count=10):
        continue

    print('resp', resp)


index = 'sit-spark-logs'
type_ = 'logs'
insert_docus(index, type_, 10000)

In [ ]:
# rollover
# create index


def create_index_rollover(index):
    """create index rollover

    like `sit-geely-logs-2016.10.31-000001`
    """
    data = {
        "aliases": {index + "-write": {}, index + "-alias": {}},
        "settings": {
            "index": {
                "number_of_shards": 7,
                "number_of_replicas": 1,
                "store.type": "niofs",
            }
        },
        "mappings": {
            "logs": {
                "_source": {"enabled": True},
                "_all": {"enabled": False},
                # "properties": {
                #     "msgid": {"type": "long"},
                #     "tag": {
                #         "type": "keyword",
                #         "doc_values": True,
                #     },
                #     "vin": {"type": "keyword"},
                #     "rowkey": {"type": "keyword"},
                #     "location": {"type": "geo_point"},
                # },
            }
        },
    }
    print("api", api + "%3C{}-%7Bnow%2Fd%7D-1%3E".format(index))
    r = requests.put(api + "%3C{}-%7Bnow%2Fd%7D-1%3E".format(index), json=data)
    #     r = requests.put(api+'{}-2018.05.25-1'.format(index), json=data)
    try:
        print(index, r.json())
    except Exception:
        print(r.text)


# create
# create_index_rollover(index="sit-speech-logs")
# create_index_rollover(index="perf-speech-logs")
# create_index_rollover(index="uat-speech-logs")
# create_index_rollover(index="prod-speech-logs")


def rollover_index(index):
    data = {
        'conditions': {
            'max_age': '1s',
        },
        "aliases": {
            "{}-alias".format(index): {},
        },
        "settings": {
            "index": {
                "number_of_shards": 2,
                "number_of_replicas": 1,
                "store.type": "niofs",
            }
        },
        "mappings": {
            "logs": {
                "_source": {
                    "enabled": True,
                },
                "properties": {
                    "level": {
                        "type": "keyword",
                        "index": "not_analyzed",
                        "doc_values": True,
                    },
                    "class": {
                        "type": "keyword",
                        "index": "not_analyzed",
                        "doc_values": True,
                    },            "message": {
                        "type": "string",
                    },
                    "datasource": {
                        "type": "keyword",
                        "index": "not_analyzed",
                        "doc_values": True,
                    }
                }
            }
        },
    }
    print('>> rollover: {}{}-write/_rollover'.format(api, index))
    r = requests.post('{}{}-write/_rollover'.format(api, index), json=data)
    print(index, r.json())


# rollover_index(index='sit-cp-logs')



In [ ]:
# reroute
url = api + "_cluster/reroute?retry_failed=true"

requests.post(url).json()